<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a3061f33d976220709b6068389af72ddb1ce2245b45383ac08f49ec073b82913
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-04 11:09:48
-------------------
qualified stocks: 89
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  7.27 L
Current:  1.41 C
-------------------
Today PnL: 56.18 K (0.4%)
Current PnL: -17.33 L (-11.65%)
CY Booked + Current PnL: -7.47 L (-5.02%)
-------------------
Total profit:  3.15 L
Total loss:  -20.47 L
-------------------
Total Booked + Current PnL: 19.51 L (16.03%)
Total Booked PnL: 36.84 L (30.27%)
Curr Year Booked PnL: 9.86 L (7.01%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 78.05 L (55.47%)
Deployed:  1.22 C
Current:  1.41 C
CAGR/XIRR %: 9.19%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DIXON,18940.15,-28.34,71.0,H-MC,7.17,143352.0,28416.0,8171.0,0.90,24.72,5.70,31.83,81.0,3.48,1.07,48.59,X40N,ATH,IT
4,APOLLOHOSP,8285.00,-13.42,63.0,H-LC,9.29,173063.0,16886.0,9207.0,1.69,10.81,5.32,16.71,22.0,1.83,1.30,30.14,X40N,BTT,HEALTHCARE
79,TTKPRESTIG,770.00,108.32,72.0,M-SC,6.64,89748.0,-11029.0,11120.0,0.48,-10.94,12.39,0.09,245.0,-0.99,0.67,17.91,OX40N,NTT,DURABLES
64,SBIN,863.00,-13.25,50.0,M-LC,4.50,215127.0,13648.0,13575.0,-0.04,6.77,6.31,13.51,60.0,1.01,1.61,20.22,XY25,NTT,BANKS
43,ITC,452.00,-36.66,60.0,H-LC,2.96,204158.0,4020.0,16863.0,1.40,2.01,8.26,10.44,4.0,0.24,1.53,7.68,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,KPIGREEN,731.35,4.03,43.0,H-SC,11.06,124173.0,-1124.0,60125.0,-0.74,-0.90,48.42,47.09,141.0,-0.02,0.93,55.41,MH,ATH,POWER
31,HAVELLS,2069.16,-0.51,60.0,H-MC,2.05,246698.0,-1033.0,76082.0,0.98,-0.42,30.84,30.30,92.0,-0.01,1.85,12.82,X40,ATH,ELECTRICAL
53,PGHH,18062.58,-30.02,57.0,H-MC,5.24,202755.0,1935.0,68187.0,0.16,0.96,33.63,34.92,80.0,0.03,1.52,5.68,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HCLTECH,1943.91,0.22,40.0,H-LC,9.86,229220.0,-12704.0,75964.0,-0.42,-5.25,33.14,26.15,8.0,-0.17,1.72,8.32,X40,ATH,IT
58,RELIANCE,1533.00,-15.17,40.0,H-LC,7.51,212581.0,2215.0,26573.0,-0.72,1.05,12.50,13.68,37.0,0.08,1.59,17.75,XY25,NTT,REFINERIES
45,JIOFIN,387.00,-1.15,42.0,H-LC,13.96,218575.0,1169.0,52327.0,-0.46,0.54,23.94,24.61,48.0,0.02,1.64,55.65,XY24,BTT,FINANCE
41,INFY,2275.00,-19.60,44.0,H-LC,11.08,313153.0,265.0,171420.0,-0.60,0.08,54.74,54.87,3.0,0.00,2.35,6.74,X40,BTT,IT
1,ABB,7934.00,-40.55,48.0,H-LC,13.23,248640.0,-12979.0,132202.0,-0.16,-4.96,53.17,45.57,7.0,-0.10,1.86,5.80,AR,NTT,ELECTRICAL


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LTIM,7230.20,-5.78,52.0,H-LC,2.14,233415.0,-12736.0,91942.0,-0.08,-5.17,39.39,32.18,16.0,-0.14,1.75,29.00,X200,ATH,IT
52,NESTLEIND,1377.00,-7.84,64.0,H-LC,2.55,283444.0,18018.0,38775.0,1.38,6.79,13.68,21.40,11.0,0.46,2.12,13.88,XY25,NTT,FMCG
43,ITC,452.00,-36.66,60.0,H-LC,2.96,204158.0,4020.0,16863.0,1.40,2.01,8.26,10.44,4.0,0.24,1.53,7.68,X40,NTT,FMCG
19,CIPLA,1795.00,-19.09,58.0,H-LC,5.39,216254.0,11754.0,29670.0,0.01,5.75,13.72,20.25,10.0,0.40,1.62,14.98,X40N,BTT,PHARMA
85,VBL,671.64,-11.16,50.0,H-LC,7.13,319766.0,3924.0,108752.0,-0.80,1.24,34.01,35.67,5.0,0.04,2.40,15.27,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,844.00,3602.70,69.0,M-SC,10.63,115404.0,-31964.0,83783.0,12.47,-21.69,72.60,35.16,236.0,-0.38,0.86,17.87,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-31.44,66.0,M-SC,10.14,102422.0,-26248.0,71542.0,6.16,-20.40,69.85,35.20,219.0,-0.37,0.77,17.56,X40,NTT,FOOTWEAR
16,CAMPUS,393.00,-23.81,62.0,M-SC,3.25,158808.0,-8967.0,64810.0,3.60,-5.34,40.81,33.28,210.0,-0.14,1.19,22.95,XY24,NTT,FOOTWEAR
22,COLPAL,3767.14,5.55,67.0,H-MC,9.10,192364.0,-19377.0,101472.0,3.54,-9.15,52.75,38.77,84.0,-0.19,1.44,14.49,XY25,ATH,FMCG
73,TANLA,1963.11,-25.67,59.0,H-SC,8.15,169741.0,-69674.0,342622.0,2.22,-29.10,201.85,114.01,133.0,-0.20,1.27,55.68,AR,ATH,IT


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-39.64,65.0,L-SC,35.35,90758.0,-22791.0,62841.0,-1.54,-20.07,69.24,35.27,268.0,-0.36,0.68,122.13,XR,NTT,HOTELS
28,GREENPANEL,537.00,193.93,43.0,M-SC,7.48,131174.0,-51904.0,130885.0,-1.21,-28.35,99.78,43.14,230.0,-0.40,0.98,23.39,XY24,NTT,MISC
82,UNITDSPR,1644.00,-12.52,52.0,H-LC,8.50,175943.0,-5816.0,41065.0,-1.05,-3.20,23.34,19.39,86.0,-0.14,1.32,4.69,X40N,NTT,BREWERIES
40,INDUSINDBK,1800.00,252.94,43.0,L-MC,11.50,43343.0,-33864.0,59259.0,-1.00,-43.86,136.72,32.89,258.0,-0.57,0.32,19.36,XR,NTT,BANKS
34,HINDZINC,730.22,17.65,57.0,M-LC,2.69,191618.0,-13458.0,126027.0,-0.92,-6.56,65.77,54.89,52.0,-0.11,1.44,15.95,X5K,ATH,METALS


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,108.32,72.0,M-SC,6.64,89748.0,-11029.0,11120.0,0.48,-10.94,12.39,0.09,245.0,-0.99,0.67,17.91,OX40N,NTT,DURABLES
80,UJJIVANSFB,60.0,84.36,46.0,H-SC,13.51,116775.0,-25704.0,45227.0,0.46,-18.04,38.73,13.70,163.0,-0.57,0.88,37.83,OX40N,NTT,BANKS
39,INDIGOPNTS,1408.0,117.63,54.0,M-SC,3.28,142749.0,-31810.0,31847.0,0.73,-18.22,22.31,0.02,221.0,-1.00,1.07,24.17,OX40N,NTT,PAINTS
46,KANSAINER,340.0,-68.16,56.0,H-SC,3.63,220167.0,-49500.0,85843.0,0.02,-18.36,38.99,13.47,138.0,-0.58,1.65,11.36,XY24,NTT,PAINTS
18,CERA,9475.0,-19.57,47.0,H-SC,2.90,114759.0,-29044.0,55796.0,1.07,-20.20,48.62,18.60,149.0,-0.52,0.86,26.64,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4998.00,-26.76,48.0,H-LC,14.51,291344.0,-54612.0,178477.0,0.06,-15.79,61.26,35.80,1.0,-0.31,2.18,3.21,X40,BTT,IT
41,INFY,2275.00,-19.60,44.0,H-LC,11.08,313153.0,265.0,171420.0,-0.60,0.08,54.74,54.87,3.0,0.00,2.35,6.74,X40,BTT,IT
43,ITC,452.00,-36.66,60.0,H-LC,2.96,204158.0,4020.0,16863.0,1.40,2.01,8.26,10.44,4.0,0.24,1.53,7.68,X40,NTT,FMCG
85,VBL,671.64,-11.16,50.0,H-LC,7.13,319766.0,3924.0,108752.0,-0.80,1.24,34.01,35.67,5.0,0.04,2.40,15.27,X40N,ATH,FMCG
1,ABB,7934.00,-40.55,48.0,H-LC,13.23,248640.0,-12979.0,132202.0,-0.16,-4.96,53.17,45.57,7.0,-0.10,1.86,5.80,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,20.37,187600.0,36687.0,20617.0,0.21,24.31,10.99,37.97,93.0,1.78,1.41,55.77,AR,ATH,AUTO
14,BLUESTARCO,2326.38,14.99,69.0,H-MC,3.44,198520.0,33850.0,34126.0,1.25,20.56,17.19,41.28,89.0,0.99,1.49,30.18,X40N,ATH,AC
25,DIXON,18940.15,-28.34,71.0,H-MC,7.17,143352.0,28416.0,8171.0,0.90,24.72,5.70,31.83,81.0,3.48,1.07,48.59,X40N,ATH,IT
26,DMART,5201.00,-5.97,71.0,H-LC,14.93,234916.0,50435.0,19944.0,0.75,27.34,8.49,38.14,38.0,2.53,1.76,40.84,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2326.38,14.99,69.0,H-MC,3.44,198520.0,33850.0,34126.0,1.25,20.56,17.19,41.28,89.0,0.99,1.49,30.18,X40N,ATH,AC
86,VOLTAS,1918.49,1.58,64.0,H-MC,4.52,213000.0,21258.0,74763.0,-0.72,11.09,35.10,50.08,99.0,0.28,1.60,18.64,XY25,ATH,AC
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,20.37,187600.0,36687.0,20617.0,0.21,24.31,10.99,37.97,93.0,1.78,1.41,55.77,AR,ATH,AUTO
23,DABUR,735.00,-0.59,69.0,H-MC,4.35,216177.0,19149.0,70474.0,1.99,9.72,32.60,45.49,102.0,0.27,1.62,24.03,XY24,BTT,FMCG
17,CAMS,5211.76,-6.49,47.0,H-SC,5.53,106400.0,4396.0,39528.0,-0.09,4.31,37.15,43.06,122.0,0.11,0.80,22.16,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,20.37,187600.0,36687.0,20617.0,0.21,24.31,10.99,37.97,93.0,1.78,1.41,55.77,AR,ATH,AUTO
51,NATIONALUM,247.44,-44.49,72.0,H-MC,0.25,110282.0,9778.0,20865.0,0.46,9.73,18.92,30.49,79.0,0.47,0.83,45.29,MH,ATH,METALS
87,WHIRLPOOL,2270.00,-39.28,63.0,M-SC,4.59,102698.0,11200.0,67555.0,1.81,12.24,65.78,86.07,223.0,0.17,0.77,49.14,XR,NTT,DURABLES
38,INDIAMART,4850.92,-51.47,47.0,H-SC,12.49,135489.0,12153.0,121615.0,1.13,9.85,89.76,108.45,119.0,0.10,1.02,34.41,AR,ATH,MISC
88,WIPRO,420.00,-12.43,47.0,M-LC,6.93,153636.0,2691.0,106762.0,-0.71,1.78,69.49,72.51,53.0,0.03,1.15,7.91,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SIS,528.00,2238.34,53.0,H-SC,5.74,62908.0,-18182.0,26849.0,-0.58,-22.42,42.68,10.69,156.0,-0.68,0.47,27.49,OX40N,NTT,MISC
62,SAMMAANCAP,326.00,-172.73,68.0,M-SC,6.68,83346.0,-18864.0,112250.0,0.22,-18.46,134.68,91.37,208.0,-0.17,0.62,35.30,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-2.99,48.0,H-SC,15.92,90343.0,-10620.0,105150.0,-0.37,-10.52,116.39,93.63,148.0,-0.10,0.68,34.56,SR,ATH,CHEMICALS
21,COFFEEDAY,80.00,-39.64,65.0,L-SC,35.35,90758.0,-22791.0,62841.0,-1.54,-20.07,69.24,35.27,268.0,-0.36,0.68,122.13,XR,NTT,HOTELS
50,MASFIN,398.61,-16.89,56.0,H-SC,13.29,94305.0,-3675.0,25274.0,0.59,-3.75,26.80,22.05,152.0,-0.15,0.71,37.08,XR,ATH,FINANCE


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-39.64,65.0,L-SC,35.35,90758.0,-22791.0,62841.0,-1.54,-20.07,69.24,35.27,268.0,-0.36,0.68,122.13,XR,NTT,HOTELS
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,20.37,187600.0,36687.0,20617.0,0.21,24.31,10.99,37.97,93.0,1.78,1.41,55.77,AR,ATH,AUTO
57,RELAXO,1176.00,-29.81,66.0,H-SC,5.36,65126.0,-50412.0,85400.0,1.88,-43.63,131.13,30.28,136.0,-0.59,0.49,28.04,X40N,NTT,FOOTWEAR
26,DMART,5201.00,-5.97,71.0,H-LC,14.93,234916.0,50435.0,19944.0,0.75,27.34,8.49,38.14,38.0,2.53,1.76,40.84,X40N,NTT,FMCG
14,BLUESTARCO,2326.38,14.99,69.0,H-MC,3.44,198520.0,33850.0,34126.0,1.25,20.56,17.19,41.28,89.0,0.99,1.49,30.18,X40N,ATH,AC


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.89
1,25,43.86
2,50,73.97


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.84
LC    36.31
MC    25.86
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.72
X40      15.68
X40N     14.46
XY25     10.06
AR        9.50
XR        9.07
OX40N     7.46
MH        1.76
X200      1.75
X5K       1.44
SR        1.11
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.03
H-MC    22.86
H-SC    22.59
M-SC    13.65
M-LC     7.20
M-MC     2.68
L-SC     1.60
L-LC     1.08
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.05,1.68,32.15
IT,13.43,-15.21,72.34
FINANCE,7.40,-22.14,67.17
BANKS,7.37,-15.89,65.10
PAINTS,6.06,-14.19,39.75
MISC,5.86,-17.61,85.02
ELECTRICAL,5.73,-7.57,66.86
AUTO,4.76,-6.46,52.78
AC,3.82,4.89,27.32


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2764566.0,22
AR,1212343.0,10
XR,1081641.0,13
X40,875987.0,10
X40N,504602.0,11
OX40N,464097.0,10
XY25,407610.0,7
SR,195749.0,2
X5K,126027.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2760305.0,27
M-SC,1422807.0,17
H-LC,1380335.0,16
H-MC,1155285.0,17
M-LC,418827.0,5
M-MC,320530.0,2
L-SC,248004.0,3
L-MC,59259.0,1
L-LC,40202.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,925289.0,6
M-SC,XY24,845889.0,7
H-SC,AR,662260.0,5
H-LC,X40,591454.0,5
H-SC,XR,522853.0,6
H-MC,XY24,448068.0,4
H-LC,AR,370626.0,2
M-MC,XY24,320530.0,2
H-MC,X40,212991.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 43.0 seconds
